# News sentiment


In [6]:
!pip install GoogleNews



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 13.4 MB/s eta 0:00:00


In [1]:
import vertexai

PROJECT_ID = "ml-demo-384110"  # @param {type:"string"}
REGION = "us-central1"

vertexai_text_bison ="text-bison@001"

vertexai.init(project=PROJECT_ID, location=REGION)

In [2]:
import logging
logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)


In [3]:
import pandas as pd

from vertexai.preview.language_models import TextGenerationModel


In [9]:
generation_model = TextGenerationModel.from_pretrained(vertexai_text_bison)

def llm(prompt,         max_output_tokens=1024,
        temperature=0,
        top_p=0.1,
        top_k=1,        
):
    result = generation_model.predict(
        prompt=prompt,
        max_output_tokens=max_output_tokens,
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,        
    )
    return result

### URL and prompt

In [26]:
from GoogleNews import GoogleNews

def load_text(file_path=None):
    with open(file_path) as f:
        content = f.readlines()
    return content

def predict(text, name):
    prompt = f"""Is the sentiment positive or negative towards {name} based on the article: {text}"""
#    prompt = f"""Get a float number between 1 and -1 describe the sentiment positive (1) or negative (-1) towards {name} based on the article: {text}"""    
    resp = llm(prompt, max_output_tokens=1024)
    return resp #["predictions"]#[0]["content"]

def query_news(name):
    news_agent = GoogleNews(lang='en')
    news_agent.enableException(True)
    news_agent.get_news(name)
    if len(news_agent.results()) == 0:
        print("no recent news about this person")
        exit()
    nums_to_return = min(5, len(news_agent.results()))
    return news_agent.results()[:nums_to_return]


In [28]:
name = "Macron"
articles = query_news(name)
print(f"the most recent five news about {name} from google news based on news titles")
for article in articles:
    title = article.get("title")
    body = article.get("body")
    sentiment = predict(title, name)
    sentiment_body = predict(body, name)
    print(f"{sentiment} vs {sentiment_body} for article '{title}'")

the most recent five news about Macron from google news based on news titles
positive vs The article is about Macron's visit to the United States. It is a neutral article. for article 'President Macron reaffirms French ownership of New Caledonia'
positive vs The article is about Macron's visit to the United States. It is a neutral article. for article 'Emmanuel Macron insists New Caledonia belongs to France out of choice'
positive vs The article is about Macron's visit to the United States. It is a neutral article. for article 'Macron urges 'sincere' talks over New Caledonia independence dispute'
positive vs The article is about Macron's visit to the United States. It is a neutral article. for article 'Emmanuel Macron warns of 'Chinese naval bases' in New Caledonia, as Antony Blinken calls out 'problematic' Beijing in Tonga'
negative vs The article is about Macron's visit to the United States. It is a neutral article. for article 'Macron says France needs return to authority ‘at every 

In [31]:
article

{'title': 'President Macron reaffirms French ownership of New Caledonia',
 'desc': None,
 'date': '3 hours ago',
 'datetime': datetime.datetime(2023, 7, 26, 17, 35, 24, 34467),
 'link': 'news.google.com/./articles/CBMiZmh0dHBzOi8vd3d3LmV1cm9uZXdzLmNvbS92aWRlby8yMDIzLzA3LzI2L3ByZXNpZGVudC1tYWNyb24tcmVhZmZpcm1zLWZyZW5jaC1vd25lcnNoaXAtb2YtbmV3LWNhbGVkb25pYdIBAA?hl=en-US&gl=US&ceid=US%3Aen',
 'img': 'https://lh3.googleusercontent.com/dDXRCHYvgF00H8KgCzNNfOyKnYuPV2iuTz2kQ6KtXFQDIqgex3VcECy9Qpn0uoYg7_6mCgAFgA=s0-h14-rp',
 'media': 'Euronews',
 'site': None}

### basic approach

In [ ]:
# Utility functions for Embeddings API with rate limiting
def rate_limit(max_per_minute):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)


class CustomVertexAIEmbeddings(VertexAIEmbeddings, BaseModel):
    requests_per_minute: int
    num_instances_per_batch: int

    # Overriding embed_documents method
    def embed_documents(self, texts: List[str]):
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

        while docs:
            # Working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch :],
            )
            chunk = self.client.get_embeddings(head)
            results.extend(chunk)
            next(limiter)

        return [r.values for r in results]

callbacks = [StdOutCallbackHandler()]
    
# LLM model
vertexai_llm = VertexAI(
    model_name=vertexai_text_bison,
    max_output_tokens=1024,
    temperature=0.2,
    top_p=0.8,
    top_k=40,
    verbose=True,
    callbacks=callbacks
)

# Chat
chat = ChatVertexAI()

# Embedding
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH = 5
embeddings = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)



    

In [ ]:

 
# function to get unique values
def unique(list1):
    x = np.array(list1)
    return np.unique(x)

def load_url(url):

    from bs4 import BeautifulSoup
    import urllib.request
    from urllib.error import URLError, HTTPError    
    parser = 'html.parser'  # or 'lxml' (preferred) or 'html5lib', if installed
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

    try:        
        req = urllib.request.Request(url, headers=headers) 
        resp = urllib.request.urlopen( req )        
    except HTTPError as e:
        # do something
        print('Error code: ', e.code)   
        return None, []
    except URLError as e:
        # do something
        print('Reason: ', e.reason)
        return None, []
    else:
        encoding = resp.info().get_param('charset')
        #print(encoding)
        soup = BeautifulSoup(resp, parser, from_encoding=encoding)
#        print(soup.get_text())

        try:
            urls = []
            for link in soup.find_all('a', href=True):
                href = link['href']
                if href.startswith("/" ) and len(href) > 1:
                    #print(href)
                    urls.append(str(url) + str(href))
            return soup, unique(urls)
        except:
            print(f"error parsing {url}")
            return soup, []

    
def prepare_page(url):
    soup , urls =load_url(url)
    if soup != None:
        docs = load_text(soup.get_text())
        if docs !=None:
            for doc in docs:
                doc.metadata["url"] = url
                doc.metadata["source"] = url
        return docs
    print("prepare_page Not found")
    return None

def prepare_pages(url, recursive = True):
        
    soup , urls =load_url(url)
    if soup != None:
    
        docs = load_text(soup.get_text())
        for doc in docs:
            doc.metadata["url"] = url
            doc.metadata["source"] = url
        if recursive == True :
            print(urls)
            for u in urls:
                docs_url = prepare_page(u)
                if docs_url !=None:
                    docs.extend(docs_url)
        return docs
    else:
        return None



In [ ]:
def load_text(html_text : str):
    from langchain.document_loaders import UnstructuredFileLoader

    from langchain.document_loaders import UnstructuredFileIOLoader

    
    from io import BytesIO

    loader = UnstructuredFileIOLoader(
        BytesIO(bytes(html_text,encoding='utf-8')), mode="elements", strategy="fast"
    )
    docs = loader.load()
    return docs


In [ ]:
url_dic = {}

In [ ]:

def chat_with_page(query, url, template) -> str:
    
    if url in url_dic:
        print(f"Exists: {url}")
        retriever = url_dic[url]
    else:
        print("Does not exist")
   
        
        documents = prepare_pages(url, False)
        if documents == None:
            print("Load document fail")
            return {'answer':{'error' : 404  }}
        else:
            # Get your splitter ready
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=50)

            # Split your docs into texts
            texts = text_splitter.split_documents(documents)

            print(f"You have {len(texts)} documents")

            # Embed your texts
            db = FAISS.from_documents(texts, embeddings)

            # Init your retriever. Asking for just 1 document back
            retriever = db.as_retriever()
            url_dic[url] = retriever

    # Create chain to answer questions
    from langchain.chains import RetrievalQA
    from langchain import PromptTemplate
    from langchain.prompts import PromptTemplate
    from langchain.chains import LLMChain
    from langchain.agents import Tool
    
    prompt = PromptTemplate(
        input_variables=["url"],
        template=template
    )

    print(80*"*")

    chain = RetrievalQAWithSourcesChain.from_chain_type(vertexai_llm, 
                                                        chain_type="stuff" , #tools= [llm_tool],
                                                        retriever=retriever, 
                                                        return_source_documents=True, verbose=True )

    result = chain(prompt.format(url=url))

    print(80*"*")
    print(result)
    print(80*"*")    
    return result


    #llm_chain = LLMChain(llm=llm, prompt=prompt)

    # initialize the LLM tool
    llm_tool = Tool(
        name='Language Model',
        func=llm_chain.run,
        description='use this tool for general purpose queries and logic'
    )
    
    # Notice "location" below, that is a placeholder for another value later





In [ ]:
prompt= """ Summarize content {url}. """

In [ ]:
url = "https://hardrunning.hubside.es"

result = chat_with_page(prompt, url, prompt)
print("")
print("LLM answer: " + result["answer"])
print("LLM sources: " + result["sources"])

Exists: https://hardrunning.hubside.es
********************************************************************************


> Entering new  chain...

> Finished chain.
********************************************************************************
{'question': ' Summarize content https://hardrunning.hubside.es. ', 'answer': "HardRunning is a running event in Malaga that will help you push your limits and achieve a great goal. You can sign up for the half marathon, an unforgettable experience that you can do with friends or by yourself. In either case, you won't regret it.\n", 'sources': 'https://hardrunning.hubside.es', 'source_documents': [Document(page_content='HardRunningInicioEl clubArtículosEventosUnirseBIENVENIDOS A HARD RUNNINGLa experiencia vivida en Málaga que hará que superes tus límites y cumplas un gran objetivoInscríbete en nuestra media maratón, una experiencia inolvidable que podrás realizar en compañía o luchando contra ti mismo. En cualquiera de los casos, no te arrepen

In [ ]:
url = "https://tintin_le_film.hubside.es"

result = chat_with_page(prompt, url, prompt)
print("")
print(result["answer"])


Does not exist
Reason:  [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:1007)
Load document fail

{'error': 404}


# 2) Aide au formulaire SEO éditeur: 
- Meta description, title, alt image
- Déjà testé Vision API
- Génération 


In [ ]:
url = "https://valerie-filuncrochet.com"

prompt = f"""You are an expert in SEO. 
Give me the 20 best tags describing this website {url} 
The result should be as an array of elements format without any parent key
"""


In [ ]:
llm(prompt)

[
  "crochet",
  "crochet patterns",
  "crochet tutorial",
  "crochet hat",
  "crochet scarf",
  "crochet blanket",
  "crochet bag",
  "crochet clothes",
  "crochet accessories",
  "crochet for beginners",
  "crochet for kids",
  "crochet for adults",
  "crochet patterns free",
  "crochet tutorial video",
  "crochet blog",
  "crochet shop",
  "crochet pattern designer",
  "crochet teacher",
  "crochet artist"
]

In [ ]:
"""You are a leading digital marketer working for a top retail organisation. 
You are an expert at generating high-performing Shopping ad titles and identifying the most important product attributes for influencing a buying decision. 
Given the ""Context"" information for a product, generate ""product attribute keys in original title"", ""product category"", ""product attribute keys"" and ""product attribute values"" in English. Sort the generated ""product attribute keys"" and ""product attribute values"" so that the most important attributes are listed first. You are not allowed to list an attribute in ""product attribute keys"" without a corresponding value in ""product attribute values"". If you encounter a size attribute with a value such as Small, Medium or Large, replace it with its abbreviation (S, M, L) and prefix it with ""Size"".
"""

In [ ]:
url

'https://hardrunning.hubside.es'

In [ ]:
from unstructured.partition.html import partition_html

elements = partition_html(url=url)
print("\n\n".join([str(el) for el in elements]))

VALERIE_FILUNCROCHET

ACCUEIL

CONTACT

KITS

LIVRE

LIVRE-FORET

LIVRE-LUMIS

LIVRE-NOEL

LIVRES - hors France

TUTORIELS

TUTORIELS-LUMIS

TUTORIELS-FLORAS-ET-MALIS

TUTORIELS-Halloween

TUTORIELS-POUPEES

TUTORIELS-CALICES

TUTORIELS-COMPAGNONS

TUTORIELS-ANNEAUX

TUTORIELS-ACCESSOIRES

PATTERNS

SACS et DECO

Mes tutoriels gratuits

shopping_bag

Valérie_filuncrochet

Créatrice de crochet _ crochet'designer

AUTEURE DE TUTORIELS

numériques et de livres en auto-édition

BOUTIQUE DE TUTORIELS DE CROCHET

PATTERN'S SHOP

www.valerie-filuncrochet.com

INFORMATIONS IMPORTANTES :

TOUS LES LIVRES SONT EN STOCK !

Sauf le livret "5 LUMIS" qui vous sera envoyé par email

(pas de réimpression prévue pour ce livret).

Valérie_filuncrochet

facebook

instagram

pinterest

22/06/2023

Retrouvez la liste des points de vente où vous pouvez trouver mes livres

Vous pouvez toujours leur demander de les commander pour vous.

Tous mes livres (ou presque) sont sur la Base des Données des Libraires (

In [ ]:
url_dic = {}

prompt = """You are an expert in SEO. 
Based on data {url} 
Give me the 20 best tags describing this website.
The result should be as an array of elements format without any parent key
"""

result = chat_with_page(prompt, url, prompt)
print("")
print(result["answer"])

Does not exist
VALERIE_FILUNCROCHETVALERIE_FILUNCROCHETACCUEILCONTACTKITSLIVRELIVRE-FORETLIVRE-LUMISLIVRE-NOELLIVRES - hors FranceTUTORIELSTUTORIELS-LUMISTUTORIELS-FLORAS-ET-MALISTUTORIELS-HalloweenTUTORIELS-POUPEESTUTORIELS-CALICESTUTORIELS-COMPAGNONSTUTORIELS-ANNEAUXTUTORIELS-ACCESSOIRESPATTERNSSACS et DECOMes tutoriels gratuitsshopping_bagValérie_filuncrochetCréatrice de crochet _ crochet'designerAUTEURE DE TUTORIELS numériques et de livres en auto-éditionBOUTIQUE DE TUTORIELS DE CROCHETPATTERN'S SHOPwww.valerie-filuncrochet.comINFORMATIONS IMPORTANTES :TOUS LES LIVRES SONT EN STOCK !Sauf le livret "5 LUMIS" qui vous sera envoyé par email (pas de réimpression prévue pour ce livret).Valérie_filuncrochetfacebookinstagrampinterest22/06/2023Retrouvez la liste des points de vente où vous pouvez trouver mes livresVous pouvez toujours leur demander de les commander pour vous.Tous mes livres (ou presque) sont sur la Base des Données des Libraires (DILICOM), vous pouvez donc les demander par

FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:

prompt = """You are an expert in SEO. 
Based on data {url} 
Give me the 20 best tags describing this website.
The result should be as an array of elements format without any parent key
"""

result = chat_with_page(prompt, url, prompt)
print("")
print(result["answer"])

Exists: https://valerie-filuncrochet.com
********************************************************************************
********************************************************************************
{'question': 'You are an expert in SEO. \nBased on data https://valerie-filuncrochet.com \nGive me the 20 best tags describing this website.\nThe result should be as an array of elements format without any parent key\n', 'answer': '', 'sources': ''}
********************************************************************************




In [ ]:
url

'https://valerie-filuncrochet.com'

In [ ]:
context = """
VALERIE_FILUNCROCHET





















shopping_bag

Valérie_filuncrochet

Créatrice de crochet _ crochet'designer



AUTEURE DE TUTORIELS

numériques et de livres en auto-édition



BOUTIQUE DE TUTORIELS DE CROCHET

PATTERN'S SHOP



www.valerie-filuncrochet.com



Picture
INFORMATIONS IMPORTANTES :

TOUS LES LIVRES SONT EN STOCK !

Sauf le livret "5 LUMIS" qui vous sera envoyé par email

(pas de réimpression prévue pour ce livret).

Valérie_filuncrochet

facebook
instagram
pinterest

22/06/2023

Retrouvez la liste des points de vente où vous pouvez trouver mes livres

Vous pouvez toujours leur demander de les commander pour vous.

Tous mes livres (ou presque) sont sur la Base des Données des Libraires (DILICOM), vous pouvez donc les demander partout.

CONTACTEZ-MOI et POINTS DE VENTE


LES KITS REVIENNENT BIENTÔT

La précommande DES KITS de Gaspard et Candice sont fermées et reviendront en août

Retrouvez mon tout premier kit sur le thème de la Fête Foraine.

Avec du matériel de qualité : coton Catania de Schachenmayr, colle et crochet (en option).

LES KITS


Un tutoriel cadeau exclusif dans chacun de vos colis !

Un tutoriel papier de 4 pages pour créer une poupée au doux nom de Féelicia sera incluse dans chaque commande sur mon site !

Ce tutoriel est exclusif et ne sera délivré autrement.

Une façon de vous remercier de votre confiance ! 🥰


"""

In [ ]:
prompt = f"""
You are an expert in SEO. 

Give me the best meta description and add some CTA to it 
Based on 
CONTEXT:
{context}
"""

llm(prompt)

In [ ]:
lang = "The result should be in french."

In [ ]:
["crochet", "tricot", "modèles de crochet", "tutoriels de crochet", "loisirs créatifs", "accessoires crochet", "décoration crochet", "cadeaux crochet", "crochet fait main", "fil à crochet", "laine à tricoter", "création de motifs crochet", "passion crochet", "artisanat crochet", "sacs au crochet", "bonnets au crochet", "écharpes au crochet", "gants au crochet", "amigurumi", "accessoires mode crochet"]

['crochet',
 'tricot',
 'modèles de crochet',
 'tutoriels de crochet',
 'loisirs créatifs',
 'accessoires crochet',
 'décoration crochet',
 'cadeaux crochet',
 'crochet fait main',
 'fil à crochet',
 'laine à tricoter',
 'création de motifs crochet',
 'passion crochet',
 'artisanat crochet',
 'sacs au crochet',
 'bonnets au crochet',
 'écharpes au crochet',
 'gants au crochet',
 'amigurumi',
 'accessoires mode crochet']

In [ ]:
prompt= """
Realize following actions:
Give me 8 tags maximum describing this website {url}
Summarize content {url}

Choose one categories for this site between: 
ONLINE SHOP, 
SERVICES, 
FASHION & BEAUTY, 
LEISURE, 
BLOG, 
PORTFOLIO & CV, 
EVENTS, 
ART & CULTURE, 
GASTRONOMY, 
ASSOCIATIONS & INSTITUTION.

The result in json format, parent keys are tags, description, category
"""

In [ ]:
url = "https://hardrunning.hubside.es"


In [ ]:
#prompt= """ Summarize content {url}. """

result = chat_with_page(prompt, url, prompt)
print("")
print(result['answer'])

Does not exist
['https://hardrunning.hubside.es/articulos'
 'https://hardrunning.hubside.es/el-club'
 'https://hardrunning.hubside.es/eventos'
 'https://hardrunning.hubside.es/unirse']
You have 6 documents
Waiting
.********************************************************************************
input_variables=['url'] output_parser=None partial_variables={} template='\nRealize following actions:\nGive me 8 tags maximum describing this website {url}\nSummarize content {url}\n\nChoose one categories for this site between: \nONLINE SHOP, \nSERVICES, \nFASHION & BEAUTY, \nLEISURE, \nBLOG, \nPORTFOLIO & CV, \nEVENTS, \nART & CULTURE, \nGASTRONOMY, \nASSOCIATIONS & INSTITUTION.\n\nThe result in json format, parent keys are tags, description, category\nIf you have no access to the webpage content, return 404\n\n' template_format='f-string' validate_template=True
********************************************************************************


> Entering new  chain...

> Finished chain.
*****

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 40.8 MB/s eta 0:00:0000:0100:01


In [ ]:
!pip uninstall -y nest_asyncio

Found existing installation: nest-asyncio 1.5.6
Uninstalling nest-asyncio-1.5.6:
  Successfully uninstalled nest-asyncio-1.5.6


In [ ]:
!pip install session-info

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.2 MB/s eta 0:00:00
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8026 sha256=5b9ee8d4967fd0cdefc64094b7c3aa8de99967d264ea9775f0fd16f2b3c5b8ff
  Stored in directory: /home/jupyter/.cache/pip/wheels/6a/aa/b9/eb5d4031476ec10802795b97ccf937b9bd998d68a9b268765a
Successfully built session-info


In [ ]:
print(result["answer"])
#print(result["error"])

In [ ]:
result["source_documents"]

[Document(page_content='HardRunningInicioEl clubArtículosEventosUnirseBIENVENIDOS A HARD RUNNINGLa experiencia vivida en Málaga que hará que superes tus límites y cumplas un gran objetivoInscríbete en nuestra media maratón, una experiencia inolvidable que podrás realizar en compañía o luchando contra ti mismo. En cualquiera de los casos, no te arrepentirás.Anímate a cumplir un sueñoMe he inscrito porque quiero tener un objetivo de esta magnitud en mi vida y demostrarme a mí misma hasta donde puedo llegar y que nada puede pararmeÁnimo Próximos eventosCaritativoSunday RunningUna carrera caritativa organizada para recoger fondos para la asociación Sunday RunningMás informaciónRetoReto HardRunningCompletar todas las plazas disponibles y volver a organizar un evento de esta magnitud al año siguiente.Más informaciónTrail runningEl HardRunning TrainingLa empresa  organiza una sesión de running en plena naturaleza varios días previos a la media maratón para cualquiera que le guste asistir. Haz

In [ ]:
result["sources"]

''

In [ ]:
prompt

'\nsummurize\n'